In [2]:
import pandas as pd
import numpy as np
import re

# 여가 관련 자료

## 일과 여가 생활 균형

In [3]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [4]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [5]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

['통계분류', '전체', '성별', '연령별', '학력별', '동거가구원수별', '혼인상태별', '가구주여부별', '종사자지위별', '가구소득별', '지역규모별', '권역별', '17개_시도별', '장애여부별']


In [6]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [7]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [8]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

Index(['통계분류', '전체', '성별', '성별', '연령별', '연령별', '연령별', '연령별', '연령별', '연령별',
       '연령별', '학력별', '학력별', '학력별', '학력별', '동거가구원수별', '동거가구원수별', '동거가구원수별',
       '혼인상태별', '혼인상태별', '혼인상태별', '가구주여부별', '가구주여부별', '종사자지위별', '종사자지위별',
       '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별',
       '가구소득별', '가구소득별', '가구소득별', '가구소득별', '가구소득별', '가구소득별', '가구소득별', '지역규모별',
       '지역규모별', '지역규모별', '권역별', '권역별', '권역별', '권역별', '권역별', '권역별', '17개_시도별',
       '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별',
       '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별',
       '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '장애여부별', '장애여부별', '장애여부별'],
      dtype='object')


In [9]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]

In [10]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

['전체', '성별', '연령별', '학력별', '동거가구원수별', '혼인상태별', '가구주여부별', '종사자지위별', '가구소득별', '지역규모별', '권역별', '17개_시도별', '장애여부별']


In [11]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [12]:
leis_동거가구원수별_합본.head(5)

통계분류(2),통계분류(2),1인,2인,3인이상
2016,표본수,1178,2452,6972
2016.1,일에 더 집중하고있다,1.3,1.5,1.7
2016.2,일에 조금 더 집중하고 있다,6.3,7.3,9
2016.3,다소 일에 집중하고 있다,16.3,16.2,21.6
2016.4,균형을 이루고 있다,34.7,35,35.6


In [13]:
leis_가구소득별_합본.head(5)

통계분류(2),통계분류(2),100만원 미만,100~200만원,200~300만원,300~400만원,400~500만원,500~600만원,600만원 이상
2016,표본수,861,1075,1594,2437,2231,1356,1048
2016.1,일에 더 집중하고있다,0.9,1.5,2.5,1.2,1.4,1.7,2.1
2016.2,일에 조금 더 집중하고 있다,4.2,8.1,10.3,9.1,8.5,8.1,8.6
2016.3,다소 일에 집중하고 있다,12.2,14.1,19.2,22.1,21,22.4,21.6
2016.4,균형을 이루고 있다,31,35.3,32.4,36.5,36.3,37.1,34.8


In [14]:
leis_전체_합본.head()

통계분류(2),통계분류(2),소계
2016,표본수,10602
2016.1,일에 더 집중하고있다,1.6
2016.2,일에 조금 더 집중하고 있다,8.6
2016.3,다소 일에 집중하고 있다,20.4
2016.4,균형을 이루고 있다,35.5


In [15]:
## 연도 리스트
year_List = [2016,2018,2019,2020,2021]

In [18]:
leis_17개_시도별_합본.index

Index(['2016', '2016.1', '2016.2', '2016.3', '2016.4', '2016.5', '2016.6',
       '2016.7', '2018', '2018.1', '2018.2', '2018.3', '2018.4', '2018.5',
       '2018.6', '2018.7', '2019', '2019.1', '2019.2', '2019.3', '2019.4',
       '2019.5', '2019.6', '2019.7', '2020', '2020.1', '2020.2', '2020.3',
       '2020.4', '2020.5', '2020.6', '2020.7', '2021', '2021.1', '2021.2',
       '2021.3', '2021.4', '2021.5', '2021.6', '2021.7'],
      dtype='object')

In [19]:
leis_17개_시도별_합본[:]['2016':'2016.2']

통계분류(2),통계분류(2),서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주
2016,표본수,1257,751,633,675,492,501,426,175,1357,462,512,590,551,532,669,719,300
2016.1,일에 더 집중하고있다,0.7,2.2,0.8,1,2,1.8,1.5,0.5,0.8,6.1,1.3,1.6,5.1,3,2.2,1.4,10.8
2016.2,일에 조금 더 집중하고 있다,7.5,7.9,6,3.4,7.4,10.5,12.7,4.5,7.8,10.4,6.3,11.4,20.3,11.1,13.2,5.1,20.9


In [21]:
# test1 = '2016.1'

# # split_list = re.split(r"[.]", test1)
# split_list = test1.split()

# for i in split_list : 
#     print(type(i))

<class 'str'>
<class 'str'>
